In [1]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import os
import math

In [2]:
base_path = r'/home/lunet/cosoc/Desktop/orion_anomaly_pipeline'
dataset = "CWRU"
split = 0.6

In [3]:
file_labels = pd.read_csv(os.path.join(base_path, "datasets_unprocessed", dataset, "labels.csv"))

In [4]:
def mk_dir(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        # directory already exists
        pass

In [5]:
# Process normal data for train test

In [6]:
normal_data = file_labels.loc[file_labels["fault"] == "normal"]

In [7]:
for index, row in normal_data.iterrows():
    new_dir_train = os.path.join(base_path, "datasets", dataset, "train", row["label"])
    new_dir_test = os.path.join(base_path, "datasets", dataset, "test", row["label"])
    mk_dir(new_dir_train)
    mk_dir(new_dir_test)
    matfile = loadmat(os.path.join(base_path, "datasets_unprocessed", dataset, str(row["file_name"]) + ".mat"))
    for i in matfile.keys():
        if "DE" in i.split("_"):
            file_name = "DE.csv"
            data = matfile.get(i)
            index_split = math.ceil((len(data))*split)
            train_data = pd.DataFrame({"timestamp": list(range(0,index_split,1)) , "value": list(data[0:index_split].flatten())})
            test_data = pd.DataFrame({"timestamp": list(range(index_split, len(data), 1)) , "value": list(data[index_split:].flatten())})
            train_data.to_csv(path_or_buf= os.path.join(new_dir_train, file_name), index=False)
            test_data.to_csv(path_or_buf=os.path.join(new_dir_test, file_name), index=False)
            
        elif "FE" in i.split("_"):
            file_name = "FE.csv"
            data = matfile.get(i)
            index_split = math.ceil((len(data))*split)
            train_data = pd.DataFrame({"timestamp": list(range(0,index_split,1)) , "value": list(data[0:index_split].flatten())})
            test_data = pd.DataFrame({"timestamp": list(range(index_split, len(data), 1)) , "value": list(data[index_split:].flatten())})
            train_data.to_csv(path_or_buf= os.path.join(new_dir_train, file_name), index=False)
            test_data.to_csv(path_or_buf=os.path.join(new_dir_test, file_name), index=False)        
        
        elif "BA" in i.split("_"):
            file_name = "BA.csv"
            data = matfile.get(i)
            index_split = math.ceil((len(data))*split)
            train_data = pd.DataFrame({"timestamp": list(range(0,index_split,1)) , "value": list(data[0:index_split].flatten())})
            test_data = pd.DataFrame({"timestamp": list(range(index_split, len(data), 1)) , "value": list(data[index_split:].flatten())})
            train_data.to_csv(path_or_buf= os.path.join(new_dir_train, file_name), index=False)
            test_data.to_csv(path_or_buf=os.path.join(new_dir_test, file_name), index=False)

    
    
    

In [8]:
### Process fault data ###

In [9]:
# creating 12k and 48k sample freq folder
hfreq_path = os.path.join(base_path, "datasets", dataset, "test", "48k")
lfreq_path = os.path.join(base_path, "datasets", dataset, "test", "12k")
mk_dir(hfreq_path)
mk_dir(lfreq_path)

In [10]:
fault_data = file_labels.loc[file_labels["fault"] != "normal"]

In [11]:
for index, row in fault_data.iterrows():
    if row["sample_freq"] == 12 and row["fault_location"] == "DE":
        directory = os.path.join(lfreq_path, "DE")
        
    elif row["sample_freq"] == 12 and row["fault_location"] == "FE":
        directory = os.path.join(lfreq_path, "FE")
            
    elif row["sample_freq"] == 48:
        directory = os.path.join(hfreq_path, "DE")
    
    else:
        print(row)
    
    matfile = loadmat(os.path.join(base_path, "datasets_unprocessed", dataset, str(row["file_name"]) + ".mat"))
    
    for i in matfile.keys():
        if "DE" in i.split("_"):
            file_name = "DE.csv"
            save_path = os.path.join(directory, row["label"])
            mk_dir(save_path)
            data = matfile.get(i)
            df = pd.DataFrame({"timestamp": list(range(0, len(data), 1)) , "value": list(data.flatten())})
            df.to_csv(path_or_buf=os.path.join(save_path, file_name), index=False)   
            
        elif "FE" in i.split("_"):
            file_name = "FE.csv"
            save_path = os.path.join(directory, row["label"])
            mk_dir(save_path)
            data = matfile.get(i)
            df = pd.DataFrame({"timestamp": list(range(0, len(data), 1)) , "value": list(data.flatten())})
            df.to_csv(path_or_buf=os.path.join(save_path, file_name), index=False)
            
        elif "BA" in i.split("_"):
            file_name = "BA.csv"
            save_path = os.path.join(directory, row["label"])
            mk_dir(save_path)
            data = matfile.get(i)
            df = pd.DataFrame({"timestamp": list(range(0, len(data), 1)) , "value": list(data.flatten())})
            df.to_csv(path_or_buf=os.path.join(save_path, file_name), index=False)        

            
    
    
        